In [92]:
from PIL import Image, ImageChops, ImageEnhance
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from sklearn.metrics import classification_report, precision_score
import pandas as pd
import os

In [93]:
def convert_to_ela_image(path, quality):
    filename = path
    resaved_filename = filename.split('.')[0] + '.resaved.jpg'
    ELA_filename = filename.split('.')[0] + '.ela.png'
    
    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'JPEG', quality=quality)
    resaved_im = Image.open(resaved_filename)
    
    ela_im = ImageChops.difference(im, resaved_im)
    
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    
    return ela_im

In [94]:
from skimage import transform
def load(filename):
    image = Image.open(filename)
    np_image = convert_to_ela_image(filename, 90)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (128, 128, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [185]:
model = load_model('saved-models/model-v1-with-casia.h5')
# model = load_model('saved-models/model-v2-with-mixed-dataset.h5')
# model = load_model('saved-models/model-v3-with-real-fake-dataset.h5')
# model = load_model('saved-models/model-v4-with-additional-hidden-layer.h5')

In [96]:
def predict_image(label):
#     print(classification_report([y_true], model.predict_classes(img)))
    if label:
        return "Tampered Image"
    return "Real Image"

In [151]:
dataset = pd.read_csv('datasets/custom-edited/custom-edited.csv')

In [152]:
X = []
Y = []
for index, row in dataset.iterrows():
    if os.path.isfile(row[0]):
        X.append(row[0])
        Y.append(row[1])
        print('done: '+str(index))
print("==== FINISHED ====")

done: 0
done: 1
done: 2
done: 3
done: 4
done: 5
done: 6
done: 7
done: 8
done: 9
done: 10
done: 11
done: 12
done: 13
done: 14
done: 15
done: 16
done: 17
done: 18
done: 19
done: 20
done: 21
done: 22
done: 23
done: 24
done: 25
done: 26
done: 27
done: 28
done: 29
done: 30
==== FINISHED ====


### BERIKUT ADALAH HASIL DARI MODEL V1 DENGAN MENGGUNAKAN TEST DARI CUSTOM DATA
#### Accuracy: 21/31

In [186]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        tampered+=1
        print('++++ TAMPERED IMAGE ++++')
    else:
        real+=1
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-11-custom.jpeg :  Real Image ( 99.99990463256836 %),  Tampered Image ( 0.00010071435099234805 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-12-custom.jpeg :  Tampered Image ( 62.07742691040039 %),  Real Image ( 37.92257606983185 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-13-custom.jpeg :  Real Image ( 99.97871518135071 %),  Tampered Image ( 0.02129188214894384 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-14-custom.jpeg :  Real Image ( 99.9356210231781 %),  Tampered Image ( 0.06438469863496721 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-15-custom.jpg :  Tampered Image ( 99.8965859413147 %),  Real Image ( 0.10341533925384283 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-16-custom.jpg :  Real Image ( 94.09681558609009 %),  Tampered Image ( 5.9031859040260315 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-17-custom.jpg :  Tampered Image ( 98.20002

In [187]:
print("BENAR: ",benar)
print("SALAH: ",salah)

BENAR:  21
SALAH:  10


### BERIKUT ADALAH HASIL DARI MODEL V2 DENGAN MENGGUNAKAN TEST DARI CUSTOM EDITED DATA
#### Accuracy: 22/31

In [183]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        print('++++ TAMPERED IMAGE ++++')
        tampered+=1
    else:
        real+=1
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-11-custom.jpeg :  Real Image ( 99.99996423721313 %),  Tampered Image ( 3.858316972582543e-05 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-12-custom.jpeg :  Tampered Image ( 61.81615591049194 %),  Real Image ( 38.18384110927582 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-13-custom.jpeg :  Real Image ( 91.62117838859558 %),  Tampered Image ( 8.378823846578598 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-14-custom.jpeg :  Real Image ( 99.97339844703674 %),  Tampered Image ( 0.026605598395690322 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-15-custom.jpg :  Tampered Image ( 61.06429100036621 %),  Real Image ( 38.93571197986603 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-16-custom.jpg :  Real Image ( 89.0152096748352 %),  Tampered Image ( 10.984792560338974 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-17-custom.jpg :  Tampered Image ( 99.89207983

In [184]:
print("BENAR: ",benar)
print("SALAH: ",salah)

BENAR:  22
SALAH:  9


### BERIKUT ADALAH HASIL DARI MODEL V3 DENGAN MENGGUNAKAN TEST DARI CUSTOM EDITED DATA
#### Accuracy: 17/31

In [180]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        tampered+=1
        print('++++ TAMPERED IMAGE ++++')
    else:
        real+=1
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-11-custom.jpeg :  Tampered Image ( 100.0 %),  Real Image ( 1.012282383703331e-12 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-12-custom.jpeg :  Tampered Image ( 99.80911612510681 %),  Real Image ( 0.19088619155809283 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-13-custom.jpeg :  Tampered Image ( 99.0831196308136 %),  Real Image ( 0.9168803691864014 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-14-custom.jpeg :  Tampered Image ( 99.99964237213135 %),  Real Image ( 0.00036005958463647403 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-15-custom.jpg :  Tampered Image ( 98.14520478248596 %),  Real Image ( 1.8547967076301575 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-16-custom.jpg :  Tampered Image ( 100.0 %),  Real Image ( 1.1602598866216045e-12 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-17-custom.jpg :  Tampered Image ( 99.9996542930603 %),  Rea

In [181]:
print("BENAR: ",benar)
print("SALAH: ",salah)

BENAR:  17
SALAH:  14


### BERIKUT ADALAH HASIL DARI MODEL V4 DENGAN MENGGUNAKAN TEST DARI CUSTOM EDITED DATA
#### Accuracy: 19/31

In [177]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        print('++++ TAMPERED IMAGE ++++')
    else:
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-11-custom.jpeg :  Real Image ( 100.0 %),  Tampered Image ( 2.6969558856136666e-11 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-12-custom.jpeg :  Real Image ( 99.16306138038635 %),  Tampered Image ( 0.8369325660169125 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-13-custom.jpeg :  Real Image ( 87.34590411186218 %),  Tampered Image ( 12.654094398021698 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-14-custom.jpeg :  Real Image ( 99.99994039535522 %),  Tampered Image ( 6.080412049414008e-05 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-15-custom.jpg :  Real Image ( 83.53263735771179 %),  Tampered Image ( 16.467367112636566 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-16-custom.jpg :  Real Image ( 73.2312560081482 %),  Tampered Image ( 26.768746972084045 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-17-custom.jpg :  Tampered Image ( 93.39426755905151 

In [178]:
print("BENAR: ",benar)
print("SALAH: ",salah)

BENAR:  19
SALAH:  12
